# 幸福久久窩的物件爬取

In [1]:
#針對99窩的物件網址下手

import requests
from bs4 import BeautifulSoup
final = []

for i in range(1,5):  #跑1~4頁
    URL = 'http://www.995home.com/newhouse.asp?page='+str(i)+'&sort='

    res = requests.get(URL)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text,'lxml').select('.newslist > li > a')   #找到連結那一項

    a = ['http://www.995home.com/'+roww['href'] for roww in soup]
           #它們的官網               串列生成式，roww['href']可以叫出網址
    [final.append(row) for row in a]
                      #把每個網址加入[]中
print(final)
print(len(final))


['http://www.995home.com/newhouse_form.asp?room_num=1175', 'http://www.995home.com/newhouse_form.asp?room_num=1170', 'http://www.995home.com/newhouse_form.asp?room_num=1162', 'http://www.995home.com/newhouse_form.asp?room_num=1135', 'http://www.995home.com/newhouse_form.asp?room_num=1094', 'http://www.995home.com/newhouse_form.asp?room_num=1021', 'http://www.995home.com/newhouse_form.asp?room_num=924', 'http://www.995home.com/newhouse_form.asp?room_num=879', 'http://www.995home.com/newhouse_form.asp?room_num=870', 'http://www.995home.com/newhouse_form.asp?room_num=868', 'http://www.995home.com/newhouse_form.asp?room_num=853', 'http://www.995home.com/newhouse_form.asp?room_num=837', 'http://www.995home.com/newhouse_form.asp?room_num=834', 'http://www.995home.com/newhouse_form.asp?room_num=832', 'http://www.995home.com/newhouse_form.asp?room_num=766', 'http://www.995home.com/newhouse_form.asp?room_num=638', 'http://www.995home.com/newhouse_form.asp?room_num=628', 'http://www.995home.com/

In [3]:
len(final)

45

In [5]:
#存檔

from bs4 import BeautifulSoup
import requests
import time
import json
import re

t1 = time.time()  #計時
jsonarray = []  #用來做jsonarray用

for i in range(len(final)):   #final包的是所有的網址
    house = {}  #存json用
    
    URL = final[i] 
    res = requests.get(URL)
    
    time.sleep(1)
    house['url'] = URL             #存json file's key-value
    house['html'] = res.text
    house['update'] = '2017-08-16'
    time.sleep(1)
    jsonarray.append(house)    #把一個個json檔存進[]裡

with open('99house_45.json','w') as f:  #輸出
    json.dump(jsonarray,f)

t2 = time.time()

print(t2-t1)

98.35583710670471


# 清洗

In [1]:
#google api 轉經緯度，郵遞區號

import requests


api_url= "https://maps.googleapis.com/maps/api/geocode/json?address={}&key="
def get_location(address):
    URL=api_url.format(address)
    res= requests.get(URL)
    location_dict = res.json()
    lat = location_dict['results'][0]['geometry']['location']['lat']
    lng = location_dict['results'][0]['geometry']['location']['lng']
    cityID = location_dict['results'][0]['address_components'][-1]['short_name']
    res.close()
    return lat,lng,cityID

address = '台北市信義區永吉路30巷'
get_location(address)  #tuple

(25.0433553, 121.5687577, '110')

In [ ]:
#預計要爬取的欄位
id: INT               物件編號  X        
cityID: INT           城市      V
url: VARCHAR(100)     物件網址  V
title:VARCHAR(100)    物件名稱  (學校名)
address:VARCHAR(200)  地址   V
pattern:VARCHAR(50)   隔間類型   V
floor:INT             物件在的樓層   V
stories:INT           總樓層   V
label:char(1)         套房或雅房   V
rent:INT              租金   V
lat:DECIMAL(15,11)    經緯度   X
lng:DECIAMAL(15,11)
sex:CHAR(1)           性別   V
space:DECIMAL(9,2)    坪數          V(但會標0坪)
smoke:CHAR(1)         抽煙    V
pet:CHAR(1)           寵物   X
cook:CHAR(1)          烹飪   X
update:date           物件更新時間   V
landlord:             房東   V
description:          物件描述   V
tmp:                  可否短期租賃   x

In [9]:
#清洗存檔

from bs4 import BeautifulSoup
import requests
import time
import json
import re

jsonarray = []
with open('99house_json/99house_original_45.json') as f:
    read_in = f.read()
    json_out = json.loads(read_in)

t1 = time.time()


def get_location(address):
    URL=api_url.format(address)
    res= requests.get(URL)
    location_dict = res.json()
    lat = location_dict['results'][0]['geometry']['location']['lat']
    lng = location_dict['results'][0]['geometry']['location']['lng']
    cityID = location_dict['results'][0]['address_components'][-1]['short_name']
    res.close()
    return lat,lng,cityID


for i in range(len(json_out)):

    soup = BeautifulSoup(json_out[i]['html'],'lxml')
    jsondict = {}  #存每個物件的key-value


    
    jsondict['url'] = json_out[i]['url']
    jsondict['update'] = json_out[i]['update']
    jsondict['rent'] = re.findall('\d+',soup.select_one('.housebox').text.replace('\n',''))[0]

    try:
        jsondict['title'] = re.findall('.+',soup.select_one('.nes_houseinfo > h2').text)[0]
    except:
        jsondict['title'] = re.findall('\:..市.+',soup.select_one('.housebox').text)[0][1:-3]

    if re.findall('\w\房',soup.select_one('.housebox').text)[0][:1] == '套':
        jsondict['label'] = "套"
    else:
        jsondict['label'] = "套"

    jsondict['address'] = re.findall('\:..市.+',soup.select_one('.housebox').text)[0][1:-3]

    #從googlemap拿經緯度
    time.sleep(1)
    api_url= "https://maps.googleapis.com/maps/api/geocode/json?address={}&key="
    get_location(address = jsondict['address'])  
    
    
    try:
        time.sleep(2)
        jsondict['cityID'] = get_location(address = jsondict['address'])[2]
        jsondict['lng'] = get_location(address = jsondict['address'])[0]
        jsondict['lat'] = get_location(address = jsondict['address'])[1]
    except:
        print(i)
        print(json_out[i]['url'])


    jsondict['pattern'] = 'NA'
    jsondict['floor'] = re.findall('\d+\樓',soup.select_one('.houseinfobox').text.replace('\n',''))[0][:-1]
    jsondict['stories'] = 'NA'
    jsondict['space'] = re.findall('\d+\坪',soup.select_one('.houseinfobox').text.replace('\n',''))[0][:-1] 
    jsondict['description'] = re.findall('\其他補充.+',soup.select_one('.houseinfobox').\
                              text.replace('\n',''))[0].split('家')[0]
    jsondict['landlord'] = re.findall('\聯絡看屋.+',soup.select_one('.houseinfobox').\
                                      text.replace('\n',''))[0][5:].replace(' ',',')
    jsondict['temp'] = 'N'
    jsondict['sex'] = 'A'
    jsondict['pet'] = 'N'
    jsondict['cook'] = 'N'
    jsondict['smoke'] = 'N'

    jsonarray.append(jsondict)

    with open('./99house_json/99house_{0}.json'.format(i),'w') as D:
        json.dump(jsondict,D)
    print('done_%d'%i)
        
        
with open('./99house_json/99house_complete_45.json','w') as F:
    json.dump(jsonarray,F)
    
t2 = time.time()
print(t2-t1)

done_0
done_1
done_2
done_3
done_4
done_5
done_6
done_7
done_8
done_9
done_10
done_11
done_12
done_13
done_14
done_15
done_16
done_17
done_18
done_19
done_20
done_21
done_22
done_23
done_24
done_25
done_26
done_27
done_28
done_29
done_30
done_31
done_32
done_33
done_34
done_35
done_36
done_37
done_38
done_39
done_40
done_41
done_42
done_43
done_44
184.83521914482117


In [10]:
#check
with open('99house_json/99house_complete_45.json') as f:
    read_in = f.read()
    json_out = json.loads(read_in)

for i in range(len(json_out)):
    print(json_out[i]['label'])

套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套
套


In [19]:
#殘酷測試區
from bs4 import BeautifulSoup
import requests
import time
import json
import re


with open('99house_json/99house_original_45.json') as f:
    read_in = f.read()
    json_out = json.loads(read_in)
    
    soup = BeautifulSoup(json_out[14]['html'],'lxml')

    print(soup.select_one('.housebox').text.replace('\n',''))
    print(re.findall('\d+',soup.select_one('.housebox').text.replace('\n',''))[0])

租金：13500 元房型:套房地址:台北市信義區永吉路30巷map
13500


In [2]:
json_out[3]['url']

'http://www.995home.com/newhouse_form.asp?room_num=1135'